# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = 'output_data/cities.csv'
file = pd.read_csv(path,encoding='UTF-8')
file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [5]:
file = file.dropna()
locations = file[["Lat", "Lng"]].astype(float)
file = file.astype({'Humidity':'float'})
hum = file['Humidity']
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=hum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
df = file[file['Max Temp'] < 80]
one = df[df['Max Temp'] > 70]
two = one[one['Wind Speed'] < 10]
three = file[file['Cloudiness'] == 0]
hoteldf = three.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hoteldf['Hotel Name'] = ''
hoteldf.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
3,3,zhuhai,0,CN,1558378755,39.0,40.71,112.04,44.60,4.47,
6,6,flinders,0,AU,1558378755,62.0,-34.58,150.85,60.01,11.41,
11,11,castro,0,CL,1558378756,70.0,-42.48,-73.76,48.20,9.17,
13,13,saskylakh,0,RU,1558378757,91.0,71.97,114.09,4.92,2.26,
26,26,colonelganj,0,IN,1558378759,14.0,27.13,81.70,94.20,5.08,


In [51]:
targetrad = 5000
targettype = 'lodging'
params = {
    "radius": targetrad,
    "types": targettype,
    "key": g_key
}
for index, row in hoteldf.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params['location'] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
#     js = response.json()
    hotels = response['results']
    try:
        hoteldf.loc[index,'Hotel Name'] = hotels[0]['name']
        print(f'The nearest hotel is {hotels[0]["name"]}')
    except:
        print('No Hotel within 5000 meters')
hoteldf.head()

No Hotel within 5000 meters
The nearest hotel is Shellharbour Resort & Conference Centre
The nearest hotel is Hotel Esmeralda
No Hotel within 5000 meters
The nearest hotel is Sanjay hotel jalpaan
The nearest hotel is Joy Inn & Suites
No Hotel within 5000 meters
The nearest hotel is Jonas Bue
The nearest hotel is Hotel San Isidro Oficial
No Hotel within 5000 meters
The nearest hotel is Hotel Escarpas do Lago Capitólio
The nearest hotel is The Irish Village
The nearest hotel is Pond Country Club
The nearest hotel is Bredasdorp Country Manor
The nearest hotel is Le Monitel
The nearest hotel is Pousada Porto Praia
The nearest hotel is Hotel Náutico de Paita
The nearest hotel is La Case Bambou
The nearest hotel is Dandeli Lake County
The nearest hotel is Yingbin Hotel
The nearest hotel is Rasoolpur Barwa
The nearest hotel is Austral Hotel
The nearest hotel is Jægtvolden Fjordhotell
The nearest hotel is Hôtel La Giralda
No Hotel within 5000 meters
The nearest hotel is Hostel Agat
The nearest

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
3,3,zhuhai,0,CN,1558378755,39.0,40.71,112.04,44.60,4.47,
6,6,flinders,0,AU,1558378755,62.0,-34.58,150.85,60.01,11.41,Shellharbour Resort & Conference Centre
11,11,castro,0,CL,1558378756,70.0,-42.48,-73.76,48.20,9.17,Hotel Esmeralda
13,13,saskylakh,0,RU,1558378757,91.0,71.97,114.09,4.92,2.26,
26,26,colonelganj,0,IN,1558378759,14.0,27.13,81.70,94.20,5.08,Sanjay hotel jalpaan


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hoteldf.iterrows()]
locations = hoteldf[["Lat", "Lng"]]

In [58]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(marker)
fig

Figure(layout=FigureLayout(height='420px'))